In [1]:
import tensorflow as tf
import Seqtoseq.Transformers as transformers
import numpy as np

# Load medical dataset

# I Need to rework this dataset format

In the translation is using a python array with each element being a tuple instead

In [2]:
diagnosis_source = []
diagnosis_target = []
classes = []

with open("./datasets_diagnosis_extra/train.dat") as f:
    for line in f:
        diagnosis_class, diagnosis_text = line.split(maxsplit=1)
        diagnosis_source.append(diagnosis_text)
        diagnosis_target.append("[start] "+diagnosis_text+" [end]")
        classes.append(int(diagnosis_class)-1)

diagnosis_source = np.expand_dims(np.array(diagnosis_source), axis=1)
diagnosis_target = np.expand_dims(np.array(diagnosis_target), axis=1)
classes = np.expand_dims(np.array(classes), axis=1)

In [3]:
diagnosis_source.shape

(14438, 1)

In [4]:
diagnosis_target.shape

(14438, 1)

In [5]:
# Test if they are equal

for i in range(diagnosis_target.shape[0]):
    assert diagnosis_source[i] == " ".join(diagnosis_target[i][0].split(" ")[1:-1])

In [6]:
diagnosis_source[0]

array(['Catheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries. The groups of patients were similar for age (left anterior descending coronary artery, 59 years; right coronary artery, 58 years; circumflex coronary artery, 62 years), patients with multivessel disease (left anterior descending coronary artery, 55%; right coronary artery, 55%; circumflex coronary artery, 64%), and patients with initial grade 0/1 antegrade flow (left anterior descending coronary artery, 79%; right coronary artery, 84%; circumflex coronary artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left anterior

In [7]:
diagnosis_target[0]

array(['[start] Catheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries. The groups of patients were similar for age (left anterior descending coronary artery, 59 years; right coronary artery, 58 years; circumflex coronary artery, 62 years), patients with multivessel disease (left anterior descending coronary artery, 55%; right coronary artery, 55%; circumflex coronary artery, 64%), and patients with initial grade 0/1 antegrade flow (left anterior descending coronary artery, 79%; right coronary artery, 84%; circumflex coronary artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left 

# Encoder / decoder

In [8]:
tf.keras.backend.clear_session()

In [9]:
# Parameters
MAX_TOKENS = 15000
MAX_SEQ = 20

EMBEDDING_SIZE = 256
LATENT_DIM = 1024

# Explanation

Seq to seq model 

Source -> Target

Source == Target autoregressive

In [10]:
lstm_return_state_simple = tf.keras.layers.LSTM(units=5, return_state=True)
lstm_return_state_bidierectional_merge = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=5, return_state=True), merge_mode="sum")
lstm_return_state_bidierectional_merge_non_state = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=5))

lstm = tf.keras.layers.LSTM(units=5, return_state=True)

Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB



In [11]:
sampleinput = np.expand_dims(np.expand_dims(np.array([1.0, 2.0, 3.0, 4.0]), axis=0), axis=1)

In [12]:
lstm_return_state_simple(sampleinput)

[<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
 array([[-0.32210338,  0.05109679, -0.3413462 , -0.13140883, -0.03864373]],
       dtype=float32)>,
 <tf.Tensor: shape=(1, 5), dtype=float32, numpy=
 array([[-0.32210338,  0.05109679, -0.3413462 , -0.13140883, -0.03864373]],
       dtype=float32)>,
 <tf.Tensor: shape=(1, 5), dtype=float32, numpy=
 array([[-0.8631031 ,  0.05448556, -0.61244166, -0.2646039 , -0.05587917]],
       dtype=float32)>]

In [13]:
lstm_return_state_bidierectional_merge(sampleinput)

[<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
 array([[-0.13813052, -0.6800886 , -0.6354073 , -0.09979478, -0.02149884]],
       dtype=float32)>,
 <tf.Tensor: shape=(1, 5), dtype=float32, numpy=
 array([[-0.17677137, -0.61797935, -0.10627368, -0.14333901, -0.01797345]],
       dtype=float32)>,
 <tf.Tensor: shape=(1, 5), dtype=float32, numpy=
 array([[-0.3110677 , -0.8732613 , -0.20648508, -0.5294663 , -0.29356617]],
       dtype=float32)>,
 <tf.Tensor: shape=(1, 5), dtype=float32, numpy=
 array([[ 0.03864086, -0.06210923, -0.5291336 ,  0.04354423, -0.0035254 ]],
       dtype=float32)>,
 <tf.Tensor: shape=(1, 5), dtype=float32, numpy=
 array([[ 0.07005788, -0.5513493 , -0.7270263 ,  0.29491407, -0.04881385]],
       dtype=float32)>]

In [14]:
lstm_return_state_bidierectional_merge_non_state(sampleinput)

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[ 0.13406432,  0.11476917, -0.07515836,  0.0324094 , -0.00403054,
        -0.01870262,  0.01584945,  0.0606403 , -0.35255352,  0.01614536]],
      dtype=float32)>

In [15]:
# input
source = tf.keras.layers.Input(shape=(None, ), dtype="int64", name="source")

# ENCODER START
embeddings = tf.keras.layers.Embedding(input_dim=MAX_TOKENS, output_dim=EMBEDDING_SIZE, mask_zero=True)(source)
encoded_source = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=LATENT_DIM), merge_mode="sum")(embeddings)
# encoded_source = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=LATENT_DIM), merge_mode="sum")(embeddings)
# A full sentence is reduce to the last state of a LSTM
# ENCODER END

# DECODER
# Encodded token and new generated token as inputs
# Reverse process like the conv1DTranspose with the segmentation model
target = tf.keras.layers.Input(shape=(None, ), dtype="int64", name="target")

#learn a representation from the target (Vocabsize, latent_dim)
#Should this be MAX_TOKENS+1 for the END?
latent_space = tf.keras.layers.Embedding(input_dim=MAX_TOKENS, output_dim=EMBEDDING_SIZE, mask_zero=True)(target)

#This will return (TARGET_SHAPE, LATENT_DIM)
# decoder = tf.keras.layers.LSTM(units=LATENT_DIM, return_sequences=True)
decoder = tf.keras.layers.GRU(units=LATENT_DIM, return_sequences=True)

#TODO: FIX TO WORK WITH LSTM
# decoded_sentence = decoder(latent_space, initial_state=[encoded_source[1], encoded_source[2]])
# decoded_sentence = decoder(latent_space, initial_state=[encoded_source[0][0], encoded_source[0][1]])
decoded_sentence = decoder(latent_space, initial_state=encoded_source)

decoded_sentence = tf.keras.layers.Dropout(0.5)(decoded_sentence)

target_next_step = tf.keras.layers.Dense(units=MAX_TOKENS, activation="softmax")(decoded_sentence)
# DECODER

# CREATE MODEL ENCODER / DECODER
encoder_decoder_model = tf.keras.Model(inputs=[source, target], outputs=[target_next_step])

encoder_decoder_model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-4), metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [16]:
encoder_decoder_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 source (InputLayer)            [(None, None)]       0           []                               
                                                                                                  
 target (InputLayer)            [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    3840000     ['source[0][0]']                 
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    3840000     ['target[0][0]']                 
                                                                                              

In [17]:
import string
import re

def standarization_source(input_string):
    strip_chars = string.punctuation
    
    lowercase = tf.strings.lower(input_string)
    remove_puntuation = tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")
    remove_puntuation = tf.strings.regex_replace(remove_puntuation, "[0-9]+[.,]?[0-9]*", "[NUMBER]")
    remove_puntuation = tf.strings.regex_replace(remove_puntuation, "\[NUMBER]%", "[NUMBER_PERCENTAGE]")
    remove_puntuation = tf.strings.regex_replace(remove_puntuation, "\-+", "")

    return remove_puntuation

def standarization_target(input_string):
    strip_chars = string.punctuation
    strip_chars = strip_chars.replace("[", "")
    strip_chars = strip_chars.replace("]", "")

    lowercase = tf.strings.lower(input_string)
    remove_puntuation = tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")
    remove_puntuation = tf.strings.regex_replace(remove_puntuation, "[0-9]+[.,]?[0-9]*", "[NUMBER]")
    remove_puntuation = tf.strings.regex_replace(remove_puntuation, "\[NUMBER]%", "[NUMBER_PERCENTAGE]")
    remove_puntuation = tf.strings.regex_replace(remove_puntuation, "\-+", "")

    return remove_puntuation

In [18]:
diagnosis_source[0]

array(['Catheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries. The groups of patients were similar for age (left anterior descending coronary artery, 59 years; right coronary artery, 58 years; circumflex coronary artery, 62 years), patients with multivessel disease (left anterior descending coronary artery, 55%; right coronary artery, 55%; circumflex coronary artery, 64%), and patients with initial grade 0/1 antegrade flow (left anterior descending coronary artery, 79%; right coronary artery, 84%; circumflex coronary artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left anterior

In [19]:
diagnosis_target[0]

array(['[start] Catheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries. The groups of patients were similar for age (left anterior descending coronary artery, 59 years; right coronary artery, 58 years; circumflex coronary artery, 62 years), patients with multivessel disease (left anterior descending coronary artery, 55%; right coronary artery, 55%; circumflex coronary artery, 64%), and patients with initial grade 0/1 antegrade flow (left anterior descending coronary artery, 79%; right coronary artery, 84%; circumflex coronary artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left 

In [20]:
source_vectorizer = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS, output_sequence_length=MAX_SEQ, output_mode="int", standardize=standarization_source)

# Target will have one more character since it we need to cut one token to let the model to guess the next one
target_vectorizer = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS, output_sequence_length=MAX_SEQ+1, output_mode="int", standardize=standarization_target) 

source_vectorizer.adapt(diagnosis_source)
target_vectorizer.adapt(diagnosis_target)

def format_dataset(source, target):
    source_vect = source_vectorizer(source)
    target_vect = target_vectorizer(target)

    return(
        {
            "source":source_vect,
            # Remove the [end] token
            # Target and source are the same
            "target":target_vect[:, :-1]
        }
        #ignore [start] token and gets up to the [end] token
        # Is one token ahead of target
        ,target_vect[:, 1:]
    )

dataset = tf.data.Dataset.from_tensor_slices((diagnosis_source, diagnosis_target)).batch(256).map(format_dataset, num_parallel_calls=8).shuffle(512).prefetch(16)

2023-05-14 14:18:20.340762: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [21]:
target_vocabulary = target_vectorizer.get_vocabulary()
source_vocabulary = source_vectorizer.get_vocabulary()

# Use to decode the predicted next token
target_ditionary_vocabulary = dict(zip(range(MAX_TOKENS), target_vocabulary))
source_ditionary_vocabulary = dict(zip(range(MAX_TOKENS), source_vocabulary))

In [22]:
datapoint = next(iter(dataset))

In [23]:
source_seq_example =datapoint[0]["source"][0]

target_seq_example = datapoint[0]["target"][0]

target_next_token = datapoint[1][0]

In [24]:
" ".join([source_vocabulary[token.numpy()] for token in source_seq_example])

'endosonography of perianal and [UNK] fistula andor abscess in crohns disease [UNK] endosonography es was performed in [NUMBER] patients with'

In [25]:
" ".join([target_ditionary_vocabulary[token.numpy()] for token in target_seq_example])

'[start] endosonography of perianal and [UNK] fistula andor abscess in crohns disease [UNK] endosonography es was performed in [NUMBER] patients'

In [26]:
" ".join([target_ditionary_vocabulary[token.numpy()] for token in target_next_token])

'endosonography of perianal and [UNK] fistula andor abscess in crohns disease [UNK] endosonography es was performed in [NUMBER] patients with'

In [27]:
for inputs, targets in dataset.take(1):
    print(f"inputs source shape: {inputs['source'].shape}")
    print(f"inputs target shape: {inputs['target'].shape}")

inputs source shape: (256, 20)
inputs target shape: (256, 20)


In [28]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor="loss", filepath="./model/medical_diagnosis_lstm")
early_callback = tf.keras.callbacks.EarlyStopping(patience=10, monitor="loss")

In [29]:
with tf.device("/CPU:0"):
    encoder_decoder_model.fit(dataset, epochs=5, callbacks=[checkpoint, early_callback])

Epoch 1/5
57/57 [==============================] - ETA: 0s - loss: 8.7271 - sparse_categorical_accuracy: 0.0550

INFO:tensorflow:Assets written to: ./model/medical_diagnosis_lstm/assets


INFO:tensorflow:Assets written to: ./model/medical_diagnosis_lstm/assets


57/57 [==============================] - 120s 2s/step - loss: 8.7271 - sparse_categorical_accuracy: 0.0550
Epoch 2/5
57/57 [==============================] - ETA: 0s - loss: 6.9377 - sparse_categorical_accuracy: 0.0556

INFO:tensorflow:Assets written to: ./model/medical_diagnosis_lstm/assets


...

INFO:tensorflow:Assets written to: ./model/medical_diagnosis_lstm/assets


57/57 [==============================] - 103s 2s/step - loss: 6.8536 - sparse_categorical_accuracy: 0.0599
Epoch 5/5
57/57 [==============================] - ETA: 0s - loss: 6.8307 - sparse_categorical_accuracy: 0.0619

INFO:tensorflow:Assets written to: ./model/medical_diagnosis_lstm/assets


INFO:tensorflow:Assets written to: ./model/medical_diagnosis_lstm/assets


57/57 [==============================] - 104s 2s/step - loss: 6.8307 - sparse_categorical_accuracy: 0.0619


In [30]:
with tf.device("/CPU:0"):
    encoder_decoder_model = tf.keras.models.load_model("./model/medical_diagnosis_lstm")

2023-05-14 14:27:41.705649: W tensorflow/core/common_runtime/graph_constructor.cc:812] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-05-14 14:27:42.541446: W tensorflow/core/common_runtime/graph_constructor.cc:812] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-05-14 14:27:42.675594: W tensorflow/core/common_runtime/graph_constructor.cc:812] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-05-14 14:27:42.681649: W tensorflow/core/common_runtime/graph_constructor.cc:812] Node 'cond' has 4 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-05-14 14:27:42.693188: W tensorflow/core/common_runtime/graph_constructor.cc:812] Node 'cond' has 4 outputs but the _output_sh

In [31]:
target_vocabulary = target_vectorizer.get_vocabulary()

# Use to decode the predicted next token
ditionary_vocabulary = dict(zip(range(MAX_TOKENS), target_vocabulary))

def decode_sequence(input_sequence):

    # Initial tokens
    tokenized_input_sentence = source_vectorizer([input_sequence])

    # We start with start token
    decoded_sentence = "[start] "+input_sequence

    for i in range(MAX_SEQ):
        tokenized_target_sentence = target_vectorizer([decoded_sentence])

        next_token_predictions = encoder_decoder_model.predict([
            tokenized_input_sentence, tokenized_target_sentence
        ], verbose=0)
        
        # the output is [1, MAX_SEQ+1, MAX_TOKENS]
        #This will give 1 value for [1, MAX_SEQ, 1] <- next token index decoded from the softmax
        sample_token_index = np.argmax(next_token_predictions[0, i, :])

        # find token for that decoded token index
        sample_token = ditionary_vocabulary[sample_token_index]

        #Append the new generated token
        decoded_sentence += " "+sample_token

        #Finish before running out of tokens in the max sequence if we found the [END] token
        if sample_token == "[end]":
            break
    
    return decoded_sentence


In [33]:
with tf.device("/CPU:0"):
    decoded_cpu = decode_sequence("The patient was feeling")

decoded_cpu

'[start] The patient was feeling of of of of of of of the the the the the of of of of of the the the'

In [34]:
with tf.device("/GPU:0"):
    decoded_gpu = decode_sequence("The patient was feeling")

decoded_gpu

'[start] The patient was feeling [UNK] of of of of of of of of of of the the the the the of of of of'